In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch

In [ ]:
#Check GPU availability

if torch.cuda.is_available():
    # to use GPU
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('GPU is:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
GPU is: Tesla T4


In [ ]:
#Reading the training set

df = pd.read_csv("IMSyPP_IT_YouTube_comments_round1.csv")

In [ ]:
df.head()

df= df[0:10000]

In [ ]:
df

,ID_Commento,ID_Video,ID_Commento.1,Nome_Utente,Testo,Tipo,Target,Tono positivo,Lingua/contesto,Annotator
0,Ugw_8YcY4vl2hYO3XZJ4AaABAg,dLOOTVapOcg,Ugw_8YcY4vl2hYO3XZJ4AaABAg,Giuseppe Corona,"Il solito vizio,o moda, della sinistra di anda...",2. offensivo,8. ideologia,NaN,NaN,IMSyPP_IT_YouTube_a0.csv
1,Ugw_8YcY4vl2hYO3XZJ4AaABAg.96hs8xBdS8o96htN_xN_Qd,dLOOTVapOcg,Ugw_8YcY4vl2hYO3XZJ4AaABAg.96hs8xBdS8o96htN_xN_Qd,Giuseppe Cammarata,-*-*-*- Giuseppe se questa notizia è vera sare...,2. offensivo,NaN,NaN,contesto,IMSyPP_IT_YouTube_a0.csv
2,Ugw_8YcY4vl2hYO3XZJ4AaABAg.96hs8xBdS8o96hvHMIEBP5,dLOOTVapOcg,Ugw_8YcY4vl2hYO3XZJ4AaABAg.96hs8xBdS8o96hvHMIEBP5,Giuseppe Corona,-*-*-*- @Giuseppe Cammarata ci sarebbe da fare...,2. offensivo,1. razzismo,NaN,NaN,IMSyPP_IT_YouTube_a0.csv
3,Ugw_MSO29v5h9xcgVoB4AaABAg,dLOOTVapOcg,Ugw_MSO29v5h9xcgVoB4AaABAg,Elenamessina 03 MESSINA,SIAMO ALLA FOLLIA,2. offensivo,NaN,NaN,contesto,IMSyPP_IT_YouTube_a0.csv
4,Ugw_ZltzXANNSm669Ol4AaABAg,dLOOTVapOcg,Ugw_ZltzXANNSm669Ol4AaABAg,fabry Iris,Ma stiamo scherzandooooo. Questi sono pazzi...,2. offensivo,NaN,NaN,contesto,IMSyPP_IT_YouTube_a0.csv
...,...,...,...,...,...,...,...,...,...,...
9995,UgzmzyiqGM9bwdtsd9Z4AaABAg,V-8Sejg-V4w,UgzmzyiqGM9bwdtsd9Z4AaABAg,Raffaello Lamera,Sinistra ipocrisia allo stato puro,2. offensivo,8. ideologia,NaN,NaN,IMSyPP_IT_YouTube_a0.csv
9996,UgznWU9Tam3QyE66Uat4AaABAg,V-8Sejg-V4w,UgznWU9Tam3QyE66Uat4AaABAg,Pasquale Smaldone,Solito demagogo del c....... Dissentire sensa ...,2. offensivo,NaN,NaN,contesto,IMSyPP_IT_YouTube_a0.csv
9997,UgzO2Y5Q2vdoXUGJqf14AaABAg,V-8Sejg-V4w,UgzO2Y5Q2vdoXUGJqf14AaABAg,Vincenzo Tassone,Sei stupenda sempre aggressiva dalla parte del...,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_a0.csv
9998,UgzoGt2u5bJsHawS_dl4AaABAg,V-8Sejg-V4w,UgzoGt2u5bJsHawS_dl4AaABAg,Sofia Senci,Fratoianni li portasse a casa sua gli i clande...,2. offensivo,8. ideologia,NaN,NaN,IMSyPP_IT_YouTube_a0.csv


Some text cleaning / pre-processing (removing @s, links..)

In [ ]:
# Text processing function

import re
import string

def clean_text(text):
    # to lower case
    text = text.lower()
    # remove links
    text = re.sub('https:\/\/\S+', '', text)
    # remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # remove next line
    text = re.sub(r'[^ \w\.]', '', text)
    # remove words containing numbers
    text = re.sub('\w*\d\w*', '', text)

    return text

In [ ]:
#Cleaning the training dataset

df['Testo'] = df["Testo"].apply(lambda x: clean_text(x))

In [ ]:
df["Testo"][1]

' giuseppe se questa notizia è vera sarebbe da fare una rivoluzione a noi ci manca tutto e loro regalano  milioni alla tunisia allora sono veramente fuori di testa'

In [ ]:
#Reading the test dataset IT

df_test = pd.read_csv("IMSyPP_IT_YouTube_comments_round2.csv")

In [ ]:
df_test

,ID_Commento,ID_Video,ID_Commento.1,Nome_Utente,Testo,Tipo,Target,Tono positivo,Lingua/contesto,Annotator
0,Ugw-TFpCf7EUCVXbYD14AaABAg,_kxk_dpRHEU,Ugw-TFpCf7EUCVXbYD14AaABAg,ArcadiA,Hanno fatto benissimo,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a0.csv
1,UgxetCLmerz2Qg-hSPV4AaABAg,_kxk_dpRHEU,UgxetCLmerz2Qg-hSPV4AaABAg,RRiccardo,Si fidiamoci del prossimo per la salute di tut...,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a0.csv
2,UgxjPx9HtQGX_A53nsJ4AaABAg,_kxk_dpRHEU,UgxjPx9HtQGX_A53nsJ4AaABAg,Januainferni,Solo gli ospedali di Barletta potranno dirci q...,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a0.csv
3,UgxjPx9HtQGX_A53nsJ4AaABAg.989ezKsZFpy98AGAPUDkg4,_kxk_dpRHEU,UgxjPx9HtQGX_A53nsJ4AaABAg.989ezKsZFpy98AGAPUDkg4,G. DM77,-*-*-*- j e tu rimani a casa così sei sereno !...,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a0.csv
4,UgxjPx9HtQGX_A53nsJ4AaABAg.989ezKsZFpy98FrKAbp4a0,_kxk_dpRHEU,UgxjPx9HtQGX_A53nsJ4AaABAg.989ezKsZFpy98FrKAbp4a0,Roberto Dibenedetto,-*-*-*- Sono immuni giù un presidente del cons...,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a0.csv
...,...,...,...,...,...,...,...,...,...,...
21081,UgzmQbwRZlMM5t8hZq94AaABAg.98EFbZxCTxL98EU5SWeNnu,x56H4uflVTs,UgzmQbwRZlMM5t8hZq94AaABAg.98EFbZxCTxL98EU5SWeNnu,Pier Angelo Arrighi,-*-*-*- @Iavicoli Marco Se lo dici te allora s...,2. offensivo,11. individuo,NaN,NaN,IMSyPP_IT_YouTube_ev_a7.csv
21082,UgzmQbwRZlMM5t8hZq94AaABAg.98EFbZxCTxL98EUrOqGbmo,x56H4uflVTs,UgzmQbwRZlMM5t8hZq94AaABAg.98EFbZxCTxL98EUrOqGbmo,Iavicoli Marco,-*-*-*- @Pier Angelo Arrighi hai ragione te l'...,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a7.csv
21083,UgzmQbwRZlMM5t8hZq94AaABAg.98EFbZxCTxL98GyisKHopr,x56H4uflVTs,UgzmQbwRZlMM5t8hZq94AaABAg.98EFbZxCTxL98GyisKHopr,Gabriele S,-*-*-*- @Iavicoli Marco @Pier Angelo Arrighi l...,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a7.csv
21084,UgzmQbwRZlMM5t8hZq94AaABAg.98EFbZxCTxL98Gz0udniyY,x56H4uflVTs,UgzmQbwRZlMM5t8hZq94AaABAg.98EFbZxCTxL98Gz0udniyY,Gabriele S,-*-*-*- @Iavicoli Marco neghi quella vergogna ...,2. offensivo,12. altro,NaN,contesto,IMSyPP_IT_YouTube_ev_a7.csv


In [ ]:
#Reading the test dataset EN

df_test_en = pd.read_csv("IMSyPP_EN_YouTube_comments_evaluation_no_context.csv")

In [ ]:
#Reading the test dataset SLO

df_test_slo = pd.read_csv("IMSyPP_SI_anotacije_round2.csv")

In [ ]:
df_test_slo.head()

,ID,uporabnik,besedilo,vrsta,tarča,jezik/kontekst,Annotator
0,1285847928138014720,pengovsky,"@jkmcnk Par dni nazaj mi je nekdo razlagal, da...",0 ni sporni govor,NaN,NaN,IMSyPP anotacije round2 a0 (AS).csv
1,1236543485081198593,Selma Bizjak🇸🇮,@Andrejibert2 @MilaKuki27 @sarecmarjan Točno t...,0 ni sporni govor,NaN,NaN,IMSyPP anotacije round2 a0 (AS).csv
2,1277272167365910530,Libero,@ObjaveIzKleti 50 EUR na mesec za širjenje pro...,0 ni sporni govor,NaN,NaN,IMSyPP anotacije round2 a0 (AS).csv
3,1273597090384547840,barjanski,"@PerfidiaDonat @BrezBesed Ko bi le ""mamice opr...",0 ni sporni govor,NaN,NaN,IMSyPP anotacije round2 a0 (AS).csv
4,1255866280504168448,Janez Snoj III 🇸🇮,@sarecmarjan je obsojen na zanikanje vsega kar...,2 žalitev,10 politika/-i,NaN,IMSyPP anotacije round2 a0 (AS).csv


In [ ]:
#Cleaning the test dataset IT

df_test['Testo'] = df_test["Testo"].apply(lambda x: clean_text(x))

In [ ]:
#Cleaning the test dataset EN

df_test_en['Text'] = df_test_en["Text"].apply(lambda x: clean_text(x))

In [ ]:
df_test.head()

,ID_Commento,ID_Video,ID_Commento.1,Nome_Utente,Testo,Tipo,Target,Tono positivo,Lingua/contesto,Annotator
0,Ugw-TFpCf7EUCVXbYD14AaABAg,_kxk_dpRHEU,Ugw-TFpCf7EUCVXbYD14AaABAg,ArcadiA,hanno fatto benissimo,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a0.csv
1,UgxetCLmerz2Qg-hSPV4AaABAg,_kxk_dpRHEU,UgxetCLmerz2Qg-hSPV4AaABAg,RRiccardo,si fidiamoci del prossimo per la salute di tut...,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a0.csv
2,UgxjPx9HtQGX_A53nsJ4AaABAg,_kxk_dpRHEU,UgxjPx9HtQGX_A53nsJ4AaABAg,Januainferni,solo gli ospedali di barletta potranno dirci q...,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a0.csv
3,UgxjPx9HtQGX_A53nsJ4AaABAg.989ezKsZFpy98AGAPUDkg4,_kxk_dpRHEU,UgxjPx9HtQGX_A53nsJ4AaABAg.989ezKsZFpy98AGAPUDkg4,G. DM77,j e tu rimani a casa così sei sereno,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a0.csv
4,UgxjPx9HtQGX_A53nsJ4AaABAg.989ezKsZFpy98FrKAbp4a0,_kxk_dpRHEU,UgxjPx9HtQGX_A53nsJ4AaABAg.989ezKsZFpy98FrKAbp4a0,Roberto Dibenedetto,sono immuni giù un presidente del consiglio c...,0. appropriato,NaN,NaN,NaN,IMSyPP_IT_YouTube_ev_a0.csv


In [ ]:
#Cleaning the test dataset SLO

df_test_slo['besedilo'] = df_test_slo["besedilo"].apply(lambda x: clean_text(x))

In [ ]:
#Import XLM-R model & tokenizer (requires sentencepiece)

!pip install sentencepiece

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer

model_name = "/content/drive/MyDrive/finetuned_hate_model/xlm-r-finetuned-en"
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=4)
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)


In [ ]:
print(df["Text"][0])

corona virus chaple hill nc lab  google it and see what you discover


In [ ]:
print({x : tokenizer.encode(x, add_special_tokens=False) for x in df["Text"].values[0].split()}) #checking how tokenizer works

{'corona': [109728], 'virus': [14994], 'chaple': [107076, 133], 'hill': [130473], 'nc': [6, 8766], 'lab': [60054], 'google': [26484], 'it': [442], 'and': [136], 'see': [1957], 'what': [2367], 'you': [398], 'discover': [103882]}


In [ ]:
tokenizer(df["Text"][0])['input_ids'] # 0 is CLS, 2 is SEP

[0,
 109728,
 14994,
 107076,
 133,
 130473,
 6,
 8766,
 60054,
 26484,
 442,
 136,
 1957,
 2367,
 398,
 103882,
 2]

In [ ]:
# install matplotlib
#!pip install matplotlib

# tokenize the text feature
tokenized_feature_raw = tokenizer.batch_encode_plus(
                            # Sentences to encode
                            df["Text"].tolist(),
                            # Add '[CLS]' and '[SEP]'
                            add_special_tokens = True
                   )
# collect tokenized sentence length
token_sentence_length = [len(x) for x in tokenized_feature_raw['input_ids']]
print('max: ', max(token_sentence_length))
print('min: ', min(token_sentence_length))
# plot the distribution
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 8))
plt.hist(token_sentence_length, rwidth = 0.9)
plt.xlabel('Sequence Length', fontsize = 18)
plt.ylabel('# of Samples', fontsize = 18)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)

Longest comment --> 1802 --> exceeds max sentence length in BERT (512)

In [ ]:
df = df.dropna(subset=['Tipo'])

In [ ]:
df_test = df_test.dropna(subset=['Tipo'])

In [ ]:
df_test_en = df_test_en.dropna(subset=['Type'])

In [ ]:
features = df["Testo"].tolist()
target = df["Tipo"].tolist()

In [ ]:
features_test = df_test["Testo"].tolist()
target_test = df_test["Tipo"].tolist()

In [ ]:
features_test_en = df_test_en["Text"].tolist()
target_test_en = df_test_en["Type"].tolist()

In [ ]:
features_test_slo = df_test_slo["besedilo"].tolist()
target_test_slo = df_test_slo["vrsta"].tolist()

In [ ]:
#Tokenizing training set

MAX_LEN = 128
tokenized_feature = tokenizer.batch_encode_plus(
                            # Sentences to encode
                            features,
                            # Add '[CLS]' and '[SEP]'
                            add_special_tokens = True,
                            # Add empty tokens if len(text)<MAX_LEN
                            padding = 'max_length',
                            # Truncate all sentences to max length
                            truncation=True,
                            # Set the maximum length
                            max_length = MAX_LEN,
                            # Return attention mask
                            return_attention_mask = True,
                            # Return pytorch tensors
                            return_tensors = 'pt'
                   )

In [ ]:
#Tokenizing test set IT

MAX_LEN = 128
tokenized_feature_test = tokenizer.batch_encode_plus(
                            # Sentences to encode
                            features_test,
                            # Add '[CLS]' and '[SEP]'
                            add_special_tokens = True,
                            # Add empty tokens if len(text)<MAX_LEN
                            padding = 'max_length',
                            # Truncate all sentences to max length
                            truncation=True,
                            # Set the maximum length
                            max_length = MAX_LEN,
                            # Return attention mask
                            return_attention_mask = True,
                            # Return pytorch tensors
                            return_tensors = 'pt'
                   )

In [ ]:
#Tokenizing test set EN

MAX_LEN = 128
tokenized_feature_test_en = tokenizer.batch_encode_plus(
                            # Sentences to encode
                            features_test_en,
                            # Add '[CLS]' and '[SEP]'
                            add_special_tokens = True,
                            # Add empty tokens if len(text)<MAX_LEN
                            padding = 'max_length',
                            # Truncate all sentences to max length
                            truncation=True,
                            # Set the maximum length
                            max_length = MAX_LEN,
                            # Return attention mask
                            return_attention_mask = True,
                            # Return pytorch tensors
                            return_tensors = 'pt'
                   )

In [ ]:
#Tokenizing test set SLO

MAX_LEN = 128
tokenized_feature_test_slo = tokenizer.batch_encode_plus(
                            # Sentences to encode
                            features_test_slo,
                            # Add '[CLS]' and '[SEP]'
                            add_special_tokens = True,
                            # Add empty tokens if len(text)<MAX_LEN
                            padding = 'max_length',
                            # Truncate all sentences to max length
                            truncation=True,
                            # Set the maximum length
                            max_length = MAX_LEN,
                            # Return attention mask
                            return_attention_mask = True,
                            # Return pytorch tensors
                            return_tensors = 'pt'
                   )

In [ ]:
# install sklearn
!pip install scikit-learn

In [ ]:
# for converting labels into numeric ones

from sklearn.preprocessing import LabelEncoder


In [ ]:
#Encoding training set labels

le = LabelEncoder()
le.fit(target)
target_num = le.transform(target)

In [ ]:
#Encoding test set labels IT

le = LabelEncoder()
le.fit(target_test)
target_num_test = le.transform(target_test)

In [ ]:
#Encoding test set labels EN

le = LabelEncoder()
le.fit(target_test_en)
target_num_test_en = le.transform(target_test_en)

In [ ]:
#Encoding test set labels SLO

le = LabelEncoder()
le.fit(target_test_slo)
target_num_test_slo = le.transform(target_test_slo)

In [ ]:
set(target_num_test_slo) #labels turned into numbers (0-4)

{0, 1, 2, 3}

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Create the DataLoader for training set
train_data = TensorDataset(tokenized_feature['input_ids'], tokenized_feature['attention_mask'], torch.tensor(target_num))
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=False)

In [ ]:
# Create the DataLoader for test set IT
validation_data = TensorDataset(tokenized_feature_test['input_ids'], tokenized_feature_test['attention_mask'], torch.tensor(target_num_test))
#validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, batch_size=16, shuffle=False)

In [ ]:
# Create the DataLoader for test set EN
validation_data_en = TensorDataset(tokenized_feature_test_en['input_ids'], tokenized_feature_test_en['attention_mask'], torch.tensor(target_num_test_en))
#validation_sampler = SequentialSampler(validation_data)
validation_dataloader_en = DataLoader(validation_data_en, batch_size=16, shuffle=False)

In [ ]:
# Create the DataLoader for test set SLO
validation_data_slo = TensorDataset(tokenized_feature_test_slo['input_ids'], tokenized_feature_test_slo['attention_mask'], torch.tensor(target_num_test_slo))
#validation_sampler = SequentialSampler(validation_data)
validation_dataloader_slo = DataLoader(validation_data_slo, batch_size=16, shuffle=False)

In [ ]:
#MODEL TRAINING

from transformers import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
for epoch in range(num_epochs):
    for batch in train_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        # Print logits during training
        logits = outputs.logits
        #print("Logits during training:", logits)

        # Compute loss and perform backward pass
        loss = outputs.loss
        loss.backward()

        # Update model parameters
        optimizer.step()
        optimizer.zero_grad()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
output_model_path = "xlm-r-finetuned-en-it"
model.save_pretrained(output_model_path)
tokenizer.save_pretrained(output_model_path)

('xlm-r-finetuned-en-it/tokenizer_config.json',
 'xlm-r-finetuned-en-it/special_tokens_map.json',
 'xlm-r-finetuned-en-it/sentencepiece.bpe.model',
 'xlm-r-finetuned-en-it/added_tokens.json')

In [ ]:
#MODEL EVALUATION

from sklearn.metrics import classification_report
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set the model to evaluation mode
model.eval()

# Lists to store predictions and true labels
all_predictions = []
all_labels = []

# Use torch.no_grad() to disable gradient computation during evaluation
with torch.no_grad():
    for batch in validation_dataloader_slo:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits

        # Get predicted labels
        _, predicted_labels = torch.max(logits, dim=1)

        # Append predictions and true labels to the lists
        all_predictions.extend(predicted_labels.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Inverse transform the predicted and true labels
original_predictions = le.inverse_transform(all_predictions)
original_labels = le.inverse_transform(all_labels)

# Print or use the evaluation results as needed
print("Classification Report:")
print(classification_report(original_labels, original_predictions))

report = classification_report(original_labels, original_predictions)

# Save the results in a dictionary
results = {
    'predictions': original_predictions,
    'labels': original_labels,
    'classification_report': report
}

# Specify the filename for the pickle file
pickle_filename = 'evaluation_results_finetuned_en_on_slo.pkl'

# Save the results to a pickle file
with open(pickle_filename, 'wb') as pickle_file:
    pickle.dump(results, pickle_file)

# Print a message indicating the pickle file has been saved
print(f"Results saved in {pickle_filename}")

Classification Report:
                    precision    recall  f1-score   support

 0 ni sporni govor       0.73      0.87      0.80     13273
1 nespodobni govor       0.19      0.05      0.07       285
         2 žalitev       0.58      0.37      0.45      6373
         3 nasilje       0.04      0.01      0.02        69

          accuracy                           0.70     20000
         macro avg       0.38      0.33      0.34     20000
      weighted avg       0.67      0.70      0.67     20000

Results saved in evaluation_results_finetuned_en_on_slo.pkl


In [ ]:
# Load the results from the pickle file
with open('evaluation_results_it_en.pkl', 'rb') as pickle_file:
    loaded_results = pickle.load(pickle_file)

# Access the loaded results
loaded_predictions = loaded_results['predictions']
loaded_labels = loaded_results['labels']
loaded_classification_report = loaded_results['classification_report']

print(loaded_classification_report)


                  precision    recall  f1-score   support

  0. appropriate       0.83      0.90      0.86     15934
1. inappropriate       0.43      0.07      0.12       183
    2. offensive       0.60      0.46      0.52      5310
      3. violent       0.11      0.24      0.15        76

        accuracy                           0.78     21503
       macro avg       0.49      0.42      0.41     21503
    weighted avg       0.77      0.78      0.77     21503

